In [ ]:
import time

from calorimeter import Calorimeter, Simulation, Layer, Electron
import matplotlib.pyplot as plt
import numpy as np


## Define calorimeter
Edit the code in the next cell to define your calorimeter

In [ ]:
DEADCELLFRACTION = 0.01 # in fraction of readout channels
CALIBRATIONERROR = 0.005 # in fraction of ionisation
NOISELEVEL = 5.0  # in ionisation units


# ## An example illustrating how the code works
# 
# Create the calorimeter. In this case a sampling calorimeter with interchaging
# layers of scintillator (low density, active) and lead (high density, inactive).

mycal = Calorimeter()

lead = Layer('lead', 2.0, 0.5, 0.0)
scintillator = Layer('Scin', 0.01, 0.5, 1.0)

for i in range(40):
    mycal.add_layers([lead, scintillator])

Run a single particle through calorimeter with tracing enabled and draw the result

In [ ]:
sim = Simulation(mycal)

ionisations, cal_with_traces = sim.simulate_with_tracing(Electron(0.0, 100.0), deadcellfraction=DEADCELLFRACTION)
# Draw with particle traces
fig, ax = plt.subplots(figsize=(14, 6))
ax = cal_with_traces.draw(ax=ax, show_traces=True)
plt.tight_layout()
plt.show()

mycal.reset()

In [ ]:
calibration_energy = 10.0  # GeV

# Run a simulation of many particles to get some statistics
start_time = time.time()
ionisations = sim.simulate(Electron(0.0, calibration_energy), 250, deadcellfraction=DEADCELLFRACTION)

# Add noise
ionisations += np.random.normal(0.0, NOISELEVEL, ionisations.shape)

end_time = time.time()

elapsed_time = end_time - start_time
print(f'Simulation took {elapsed_time/250:.4f} seconds per particle')
meanionisations = np.mean(ionisations, axis=0)
rmsionisations = np.std(ionisations, axis=0)
energies = np.sum(ionisations, axis=1)

# Apply calibration error
energies *= np.random.normal(1.0, CALIBRATIONERROR, energies.shape)

rel_resolution = np.std(energies)/np.mean(energies)

thickness = sum([vol.layer.get_thickness() for vol in mycal.volumes(active=False)])
active_layers = len(mycal.volumes(active=True))

print('--- Calorimeter summary ---')

print(f'Calorimeter created with total thickness {thickness} cm, of which {active_layers} layers are active.')
if thickness > 40.0:
    print('ERROR: The calorimeter is too thick! 40cm is the maximum allowed.')
print(f'Noise level is {NOISELEVEL} ionisations per layer.')
print(f'Calibration error is {CALIBRATIONERROR*100:.1f} % per particle.')
print(f'The fraction of dead cells is {DEADCELLFRACTION*100:.2f} %.')
print(f'Relative resolution at the energy of {calibration_energy} GeV is {rel_resolution:.3f}')

calibration_factor = np.mean(energies)/calibration_energy
print(f'Calibration factor is {calibration_factor:.0f} ionisations per GeV')

# Create some plots

fig = plt.figure(figsize=(15,5))
ax1, ax2, ax3 = fig.subplots(1, 3)
zcors = [vol.z for vol in mycal.volumes(active=True)]
for i in ionisations:
    ax1.plot(zcors, i)
ax1.set_xlabel(r'$x$ [cm]')
ax1.set_ylabel(r'ionisation')

ax2.errorbar(zcors, meanionisations, yerr=rmsionisations)
ax2.set_xlabel(r'$x$ [cm]')
ax2.set_ylabel(r'ionisation (mean and spread)')

ax3.hist(energies)

plt.tight_layout()
plt.show()

In [ ]:
# # Your exercise should start here

# ## Study energy resolution
# Make a plot of the relative energy resolution, $\sigma(E)/E$ for particles between 1 and 10 GeV.
# You should be able to demonstrate that the resolution is proportional to $1/\sqrt{E}$.
# This will require that you run the simulation code above inside a loop to get number for multiple
# different energies.

particle_energies = np.logspace(0.0, 1.1, 10)


# 2 marks: Create loop to find resolutions for different energies
# 2 marks: Create plot of the relative energy resolution vs. energy
# 2 marks: Document that it shows the $1/\sqrt{E}$ behaviour through a fit or similar